In [1]:
# 1) import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import selenium

In [2]:
# 2) Create a path to chrome driver, 3) load the path into browser 4) initialize the browser
executable_path = {'executable_path': '/Users/Pariah/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
# 5) initialize the page parameters and the first site visit:
###############################################################
next_page = "https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage=0&start=0&rows=20"
page_remaining = True
page_no = 0
increment = 20
start_no = 0
# 6) make a variable that holds the url we want to scrape data from
laptops = {}

frys_laptops = []

# 7) using the browser function, visit the webpage
browser.visit(next_page)
# 8) stop the scrape process for 1.25 seconds using the time function. This allows the webpage to load in full
time.sleep(1.25)

# 9) create the html variable that allows the browser to visit the HTML elements on the webpage
html = browser.html
# 10) initialize the BeautifulSoup module by telling it to parse html from the HTML elements on the webpage.
soup = bs(html, 'html.parser')


# 11) loop through until no products are found:
###############################################################
while page_remaining == True:
    
    # 12) Data retrieval logic: 
    ###############################################################
    try:
        mothers = soup.find_all('div', class_='togrid')
        for mother in mothers:
            title = mother.find('p', class_='productDescp').get_text().strip()
            price = mother.find('label', class_='red_txt').get_text().strip()
            partial_link = mother.find('p', class_='productDescp').find('a')['href'].strip()
            base_url = "https://www.frys.com/"
            link = base_url + partial_link
            
            results = mother.find_all('div', class_='prodModel')
            for result in results:
                new_list = result.find_all('p', class_='mar-btm')
                for p in new_list:
                    dataitem = p.text.replace(" ", "")
                    di = dataitem.replace('\n', "")
                    dataitem_split = di.split(":")
                    if dataitem_split[0] == "Brand":
                        brand = dataitem_split[1]
                    if dataitem_split[0] == "UPC":
                        upc = dataitem_split[1]
                    if dataitem_split[0] == "Model":
                        model = dataitem_split[1]
                        
            # 13) create a new dictionary that will hold the data that we scrape
            dic = {
                'title': title,
                'price': price,
                'link': link,
                'upc': upc,
                'model': model,
                'brand': brand
            }
            print(dic)
            frys_laptops.append(dic)
            
    except AttributeError:
        # errors out when there are 2 prices in red (price with rebate)
        print("error found. skipping...")
    
    
    # 14) then process the Next Page logic:
    ###############################################################
    page_no += 1
    start_no = start_no + increment
    next_page = f'https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage={page_no}&start={start_no}&rows=20'

    browser.visit(next_page)
    time.sleep(1.25)

    html = browser.html
    soup = bs(html, 'html.parser') 
    
    next_page_avail = soup.find_all('p')
    for p in next_page_avail:
        message = p.text
        if message == " No products were found that matched your search":
            page_remaining = False
        
# list of dictionaries:
# frys_laptops
# create a dataframe from the list of dictionaries:
frys_df = pd.DataFrame(frys_laptops)
frys_df.head()
# bestbuy_df = pd.DataFrame(bestbuy_laptops)
# bestbuy_df.head()
# change to your username:password@localhost:5432/laptop_db:
connection = "postgres:helloworld@localhost:5432/laptop_db"
engine = create_engine(f'postgresql://{connection}')
# add the FRYS dataframe to the table:
frys_df.to_sql(name="frys_laptops", con=engine, if_exists='replace', index=False)
# add the BEST BUY dataframe to the table:
# bestbuy_df.to_sql(name="bestbuy_laptops", con=engine, if_exists='replace', index=False)
engine.table_names()

{'title': 'Dell E6230 12.5" Refurbished Laptop With Intel Ci5-3320m Processor, 4GB Memory, 250GB Hard Drive, Windows 10', 'price': '$399.00', 'link': 'https://www.frys.com/product/9907916;jsessionid=89C617C9AEF19E4E89CD3E073137B1FE.node2?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897763632', 'model': 'E6230i5-3320m', 'brand': 'Dell'}
{'title': 'Lenovo Refurbished Thinkpad Yoga 11E 11.6" Touchscreen 2-in-1 Laptop With Intel Celeron N2930 Processor, 4GB memory, 320GB HD and Windows10 Home', 'price': '$349.00', 'link': 'https://www.frys.com/product/9686093;jsessionid=89C617C9AEF19E4E89CD3E073137B1FE.node2?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897736896', 'model': 'ThinkpadYoga11E', 'brand': 'Lenovo'}
{'title': 'HP PB 6470B 14" Refurbished Laptop With Intel Ci5-3210M Processor, 8GB Memory, 500GB Hard Drive, DVD-ROM, Windows 10', 'price': '$399.00', 'link': 'https://www.frys.com/product/9800005;jsessionid=89C617C9AEF19E4E89CD3E073137B1FE.node2?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '69689775

{'title': 'Acer A315-21-90LC 15.6" Laptop With AMD A9-9420e Processor, 12GB Memory, 1TB Hard Drive, Windows 10', 'price': '$479.00', 'link': 'https://www.frys.com/product/9871315?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '193199075475', 'model': 'A315-90LCA9-9420e', 'brand': 'Acer'}
{'title': 'Dell G7790-7940GRY-PUS 17.3" Gaming Laptop With New Intel 9th Generation Intel® Core" i7-9750H Processor, 16GB Memory, 1TB Hard Drive, 256GB SSD, RTX2070 8GB Graphics, Windows 10', 'price': '$1,999.00', 'link': 'https://www.frys.com/product/9896625?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116315940', 'model': 'G7790i7-9750H', 'brand': 'Dell'}
{'title': 'HP 9470 14" Refurbished Laptop With Intel Ci5-3427U Processor, 8GB Memory, 180GB SSD, Windows 10 **90 Days Warranty**', 'price': '$299.00', 'link': 'https://www.frys.com/product/9647553?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897734533', 'model': '9470Ci5-3427U', 'brand': 'HP'}
{'title': 'HP 15-da0021cy 15.6" Refurbished Touchscreen Laptop With Intel 

{'title': 'Dell i5482-7120SLV-PUS 14" 2-in-1 Touchscreen Laptop With Intel Ci7-8565U Processor, 16GB Memory, 512GB SSD, Windows 10', 'price': '$979.00', 'link': 'https://www.frys.com/product/9851135?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116313120', 'model': 'Insp145482i7-8565U', 'brand': 'Dell'}
{'title': 'Asus C523NA-DH02 15.6" Chromebook With Intel Celeron N3350 Processor, 4GB Memory, 32GB eMMC, Chrome OS', 'price': '$243.00', 'link': 'https://www.frys.com/product/9768034?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192876099223', 'model': 'C523NA-DH02', 'brand': 'Asus'}
{'title': 'MSI GL63 8SE-013 15.6" Gaming Laptop With Intel Ci7-8750H Processor, 16GB Memory, 1TB Hard Drive + 256GB SSD, RTX2060 6GB Graphics, Windows 10', 'price': '$1,499.00', 'link': 'https://www.frys.com/product/9882645?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '824142175682', 'model': 'GL63-013i7-8750H', 'brand': 'MSI'}
error found. skipping...
{'title': 'Samsung CB Pro XE510C25 - K01US 12.3" 2-in-1 Touchscreen Chromeboo

{'title': 'Lenovo X131E 11.6" Refurbished Laptop With AMD Dual Core Processor, 4GB Memory, 320GB Hard', 'price': '$117.55', 'link': 'https://www.frys.com/product/9369530?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '720089838331', 'model': 'LenovoX131E', 'brand': 'Lenovo'}
{'title': '* Refurbished* Rose Gold 15.6" Measured Diagonal 8GB, 2TB, Touch, Win 10 ** 90 Days Warranty **', 'price': '$399.00', 'link': 'https://www.frys.com/product/9548172?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192018104921', 'model': 'HP15-bs027', 'brand': 'HP'}
{'title': 'Dell i5579-7961GRY 2 in 1 15.6" Touchscreen Laptop With Intel Ci7-8550U Processor,16GB Memory, 512GB SSD and Windows 10', 'price': '$899.00', 'link': 'https://www.frys.com/product/9340880?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116275961', 'model': 'i5579-7961GRY-PUS', 'brand': 'Dell'}
{'title': 'HP 14-cb010nr 14" Streambook With Intel Celeron N3060 Processor, 4GB Memory, 32GB eMMC, Windows 10 Home in S mode - Aqua Blue, Office 365 Personal 1Yr.', 'pri

{'title': 'MSI GF63 15.6" Gaming Laptop With Intel Core i7-8750H Processor, 16GB Memory, 1TB+256GB SSD, GTX1050Ti 4GB Graphics and Windows 10', 'price': '$1,049.00', 'link': 'https://www.frys.com/product/9698003?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '824142168608', 'model': 'GF638RD-088', 'brand': 'MSI'}
{'title': 'Dell i5770-3628GLD-PUS 17.3" Laptop With Intel Ci3-7130U Processor, 8GB Memory, 1TB Hard Drive, Windows 10', 'price': '$399.00', 'link': 'https://www.frys.com/product/9852545?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116312628', 'model': 'Insp155570i3-7130U', 'brand': 'Dell'}
{'title': 'Dell i5575-A789SLV-PUS 15.6" Laptop With AMD Ryzen 3 2200U Processor, 8GB Memory, 1TB Hard Drive, Windows 10', 'price': '$364.21', 'link': 'https://www.frys.com/product/9852625?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116312789', 'model': 'Insp155575R32200U', 'brand': 'Dell'}
{'title': 'Dell 15 KBL-R, 15.6" Laptop With Intel® Core™ i7 Processor, 8GB Memory,Intel® UHD Graphics 620 Graphics Card,

{'title': 'Lenovo Ideapad 700 Intel Core i7-6700HQ, 15.6",16GB Memory, 1TB Hard Drive + 128GB SSD, Windows 10 Laptop', 'price': '$499.00', 'link': 'https://www.frys.com/product/8861792?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '190576355564', 'model': 'Ideapad700', 'brand': 'Lenovo'}
{'title': 'Lenovo Flex 5 15.6" Touchscreen Laptop With Intel Ci7-7500U Processor, 16GB,1TB+256SSD, GeForce 940MX 2GB, Win10', 'price': '$599.00', 'link': 'https://www.frys.com/product/9279769?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '191376576883', 'model': 'Flex5', 'brand': 'Lenovo'}
{'title': 'Dell Insp.15 15.6" Laptop With Intel Ci3-7130U Processor, 8GB Memory, 1TB Hard Drive, DVDRW Windows 10', 'price': '$261.97', 'link': 'https://www.frys.com/product/9495651?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116277378', 'model': 'i3567-3378BLK-PUS', 'brand': 'Dell'}
{'title': 'Microsoft Surface 13.5" laptop with Intel Core i5 Processor,8GB, 256GB, Touch,Windows 10S DAG-00001-Platinum Color', 'price': '$699.00', 'link': 